In [2]:
from datetime import datetime
import itertools
import pandas as pd
import numpy as np
import requests
import json
from tqdm.auto import tqdm

/home/cody/Desktop/dylon/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
solar_home_data = pd.read_csv("data/2010-2011 Solar home electricity data.csv")

In [3]:
np.unique([(solar_home_data.Customer == i).sum() for i in solar_home_data.Customer.unique()])

array([ 730, 1095])

Each customer has either two or three types of recordings for each day of the year. The recordings correspond to the consumption categories.

In [4]:
solar_home_data

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,GC,1-Jul-10,0.303,0.471,0.083,0.121,0.361,...,0.495,0.540,0.406,0.543,0.495,0.216,0.378,0.128,0.078,0.125
1,1,3.78,2076,CL,1-Jul-10,1.250,1.244,1.256,0.744,0.019,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,3.78,2076,GC,2-Jul-10,0.116,0.346,0.122,0.079,0.120,...,0.262,1.100,1.012,0.817,0.526,0.335,0.402,0.142,0.120,0.111
4,1,3.78,2076,CL,2-Jul-10,1.238,1.238,1.256,1.250,0.169,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269730,300,3.36,2086,CL,29-Jun-11,2.136,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.490
269731,300,3.36,2086,GG,29-Jun-11,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
269732,300,3.36,2086,GC,30-Jun-11,1.306,0.851,0.113,0.151,0.104,...,0.948,0.917,0.872,0.832,0.863,0.789,0.280,0.224,0.259,0.774
269733,300,3.36,2086,CL,30-Jun-11,0.904,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.528


In [5]:
solar_home_data.Postcode.unique()

array([2076, 2259, 2263, 2220, 2261, 2230, 2126, 2262, 2026, 2218, 2265,
       2208, 2229, 2085, 2250, 2283, 2305, 2285, 2300, 2093, 2025, 2041,
       2303, 2082, 2284, 2137, 2048, 2282, 2205, 2224, 2021, 2290, 2010,
       2135, 2251, 2162, 2227, 2295, 2190, 2029, 2204, 2031, 2039, 2087,
       2074, 2330, 2291, 2119, 2099, 2066, 2100, 2081, 2320, 2134, 2280,
       2289, 2294, 2159, 2008, 2209, 2203, 2278, 2079, 2228, 2044, 2046,
       2111, 2077, 2018, 2325, 2324, 2196, 2231, 2120, 2260, 2292, 2293,
       2086, 2207, 2096, 2047, 2233, 2034, 2092, 2264, 2037, 2296, 2286,
       2200, 2219, 2315, 2211, 2154, 2088, 2198, 2326, 2084, 2321, 2281,
       2216])

In [9]:
postcode_data = pd.read_csv("data/AU.csv")

In [10]:
postcode_data

,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,AU,200,Australian National University,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2777,149.1189,1.0
1,AU,221,Barton,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.3049,149.1412,4.0
2,AU,2540,Jervis Bay,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.1499,150.6969,4.0
3,AU,2540,Wreck Bay,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.1648,150.6908,4.0
4,AU,2540,Hmas Creswell,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.0280,150.5501,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16868,AU,6989,Maddington,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0500,115.9833,4.0
16869,AU,6990,Gosnells,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0810,116.0054,4.0
16870,AU,6991,Kelmscott,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1243,116.0259,4.0
16871,AU,6992,Armadale,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1461,116.0093,4.0


In [11]:
postcode_data = postcode_data.drop(postcode_data[postcode_data['postal code'].duplicated()].index)

In [9]:
def add_coord(datarow):
    postcode_row = postcode_data[datarow.Postcode == postcode_data['postal code']]
    coords = pd.Series({"longitude": postcode_row.longitude.item(), "latitude": postcode_row.latitude.item()})
    return pd.concat((datarow, coords))


solar_home_data = solar_home_data.apply(add_coord, axis=1)

Convert to net energy, make the dataset hourly, and put it in long form

In [12]:
new_data = []
for customer in tqdm(solar_home_data.Customer.unique()):
    for date in solar_home_data.date.unique():
        rows = solar_home_data[(solar_home_data.Customer == customer) & (solar_home_data.date == date)]
        consumption_rows = rows[(rows['Consumption Category'] == "GC") | (rows['Consumption Category'] == "CL")]
        generation_row = rows[rows['Consumption Category'] == "GG"]
        time_columns = rows.columns[5:-2]
        generation_row.loc[:, time_columns] = consumption_rows[time_columns].sum() - generation_row[time_columns]
        new_row = generation_row.drop(columns=["Consumption Category"])
        new_data.append(new_row)
solar_home_data = pd.concat(new_data)

100%|██████████| 300/300 [33:02<00:00,  6.61s/it]


In [17]:
# Get rid of half hour intervals
time_columns = solar_home_data.columns[4:-2]
for half_hour, hour in zip(time_columns[::2], time_columns[1::2]):
    solar_home_data[hour] += solar_home_data[half_hour]
solar_home_data = solar_home_data.drop(columns=time_columns[::2])
solar_home_data

,Customer,Generator Capacity,Postcode,date,1:00,2:00,3:00,4:00,5:00,6:00,...,17:00,18:00,19:00,20:00,21:00,22:00,23:00,0:00,longitude,latitude
2,1,3.78,2076,1-Jul-10,3.268,2.204,1.709,0.289,1.342,0.365,...,3.518,2.656,0.795,1.035,0.949,0.711,0.506,1.278,151.0974,-33.7232
5,1,3.78,2076,2-Jul-10,2.938,2.707,0.990,1.151,0.658,1.118,...,0.636,2.373,0.960,1.362,1.829,0.861,0.544,1.319,151.0974,-33.7232
8,1,3.78,2076,3-Jul-10,4.613,3.929,2.403,1.414,1.345,0.456,...,1.594,2.149,2.212,0.828,1.056,0.744,0.280,1.595,151.0974,-33.7232
11,1,3.78,2076,4-Jul-10,2.732,2.714,1.423,0.350,1.526,0.238,...,3.333,3.831,2.768,2.079,0.841,1.091,0.521,1.608,151.0974,-33.7232
14,1,3.78,2076,5-Jul-10,3.270,2.961,1.325,0.660,1.227,0.457,...,2.912,1.740,1.373,1.849,0.680,1.203,0.257,1.532,151.0974,-33.7232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269722,300,3.36,2086,26-Jun-11,6.371,2.540,0.830,0.534,0.204,0.193,...,0.236,1.710,2.608,2.670,1.082,1.059,0.887,3.752,151.2232,-33.7482
269725,300,3.36,2086,27-Jun-11,6.668,1.764,0.245,0.241,0.237,0.442,...,0.134,0.599,3.915,1.885,1.490,1.167,1.094,3.694,151.2232,-33.7482
269728,300,3.36,2086,28-Jun-11,6.098,0.215,0.204,0.206,0.207,2.037,...,2.013,2.203,3.302,1.883,1.686,1.508,0.297,3.645,151.2232,-33.7482
269731,300,3.36,2086,29-Jun-11,4.076,0.382,0.234,0.230,0.257,2.093,...,0.223,2.484,3.338,2.392,2.341,2.170,1.490,3.906,151.2232,-33.7482


In [18]:
id_columns = solar_home_data.columns[:4].tolist() + solar_home_data.columns[-2:].tolist()
time_columns = solar_home_data.columns[4:-2]
solar_home_data = pd.melt(solar_home_data, id_vars=id_columns, value_vars=time_columns)
solar_home_data

,Customer,Generator Capacity,Postcode,date,longitude,latitude,variable,value
0,1,3.78,2076,1-Jul-10,151.0974,-33.7232,1:00,3.268
1,1,3.78,2076,2-Jul-10,151.0974,-33.7232,1:00,2.938
2,1,3.78,2076,3-Jul-10,151.0974,-33.7232,1:00,4.613
3,1,3.78,2076,4-Jul-10,151.0974,-33.7232,1:00,2.732
4,1,3.78,2076,5-Jul-10,151.0974,-33.7232,1:00,3.270
...,...,...,...,...,...,...,...,...
2627995,300,3.36,2086,26-Jun-11,151.2232,-33.7482,0:00,3.752
2627996,300,3.36,2086,27-Jun-11,151.2232,-33.7482,0:00,3.694
2627997,300,3.36,2086,28-Jun-11,151.2232,-33.7482,0:00,3.645
2627998,300,3.36,2086,29-Jun-11,151.2232,-33.7482,0:00,3.906


In [19]:
solar_home_data = solar_home_data.rename(columns={"value": "Net Energy", "date": "Datetime", "longitude": "Longitude", "latitude": "Latitude"})
solar_home_data['date'] += " " + solar_home_data['variable']
solar_home_data = solar_home_data.drop(columns='variable')
solar_home_data['date'] = solar_home_data['date'].map(lambda x: datetime.strptime(x, "%d-%b-%y %H:%M").isoformat(timespec="minutes"))
solar_home_data

,Customer,Generator Capacity,Postcode,date,longitude,latitude,variable,Net Energy
0,1,3.78,2076,2010-07-01T01:00,151.0974,-33.7232,1:00,3.268
1,1,3.78,2076,2010-07-02T01:00,151.0974,-33.7232,1:00,2.938
2,1,3.78,2076,2010-07-03T01:00,151.0974,-33.7232,1:00,4.613
3,1,3.78,2076,2010-07-04T01:00,151.0974,-33.7232,1:00,2.732
4,1,3.78,2076,2010-07-05T01:00,151.0974,-33.7232,1:00,3.270
...,...,...,...,...,...,...,...,...
2627995,300,3.36,2086,2011-06-26T00:00,151.2232,-33.7482,0:00,3.752
2627996,300,3.36,2086,2011-06-27T00:00,151.2232,-33.7482,0:00,3.694
2627997,300,3.36,2086,2011-06-28T00:00,151.2232,-33.7482,0:00,3.645
2627998,300,3.36,2086,2011-06-29T00:00,151.2232,-33.7482,0:00,3.906


 Add weather data

In [12]:
# Download weather data
for postcode in tqdm(solar_home_data.Postcode.unique()):
    postcode_row = postcode_data[postcode == postcode_data['postal code']]
    latitude, longitude = postcode_row.latitude.item(), postcode_row.longitude.item()
    r = requests.get(f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date=2010-07-01&end_date=2011-06-30&hourly=temperature_2m,precipitation,cloudcover")
    if r.ok:
        with open(f"data/weather/{postcode}.json", 'w') as f:
            json.dump(r.json(), f)
    else:
        print(f"Postcode {postcode} weather data request failed")

100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


In [7]:
solar_home_data['Temperature'] = 0
solar_home_data['Precipitation'] = 0
solar_home_data['Cloud cover'] = 0
for postcode in (pbar := tqdm(solar_home_data.Postcode.unique())):
    with open(f"data/weather/{postcode}.json", 'r') as f:
        weather = json.load(f)
    for i, time in enumerate(weather['hourly']['time']):
        pbar.set_postfix_str(f"{time=}")
        row_idx = (solar_home_data.Postcode == postcode) & (solar_home_data.Datetime == time)
        solar_home_data.loc[row_idx, 'Temperature'] = weather['hourly']['temperature_2m'][i]
        solar_home_data.loc[row_idx, 'Precipitation'] = weather['hourly']['precipitation'][i]
        solar_home_data.loc[row_idx, 'Cloud cover'] = weather['hourly']['cloudcover'][i]

  0%|          | 0/100 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/weather/2076.json'

In [5]:
solar_home_data = pd.read_csv("data/solar_home_2010-2011_without_weather.csv")

In [13]:
solar_home_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628000 entries, 0 to 2627999
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   Customer            int64  
 2   Generator Capacity  float64
 3   Postcode            int64  
 4   Datetime            object 
 5   Longitude           float64
 6   Latitude            float64
 7   Net Energy          float64
 8   Temperature         int64  
 9   Precipitation       int64  
 10  Cloud cover         int64  
dtypes: float64(4), int64(6), object(1)
memory usage: 220.6+ MB


In [18]:
with open("data/weather/2010.json", "r") as f:
    weather = json.load(f)

In [26]:
import sys

sys.getsizeof(weather)

272

In [17]:
solar_home_data.drop(columns=["Temperature", "Precipitation", "Cloud cover", "Unnamed: 0"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628000 entries, 0 to 2627999
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Customer            int64  
 1   Generator Capacity  float64
 2   Postcode            int64  
 3   Datetime            object 
 4   Longitude           float64
 5   Latitude            float64
 6   Net Energy          float64
dtypes: float64(4), int64(2), object(1)
memory usage: 140.4+ MB


In [29]:
solar_home_data.to_csv("data/solar_home_2010-2011.csv")